# VAEGAN NEGATIVE BINOMIAL

> The VAEGAN model, takes an Variational Encoder, Variational Decoder and Classifier model as inputs. Uses a negative binomial as the latent variable rather than Gausian.

In [ ]:
#| hide
from nbdev.showdoc import *

KeyboardInterrupt: 

In [ ]:
#| default_exp Models.VAEGAN_NEG_BI

In [ ]:
#| export
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import NegativeBinomial

class VAEGAN_NEG_BI(nn.Module):
    def __init__(self, encoder, decoder, classifier):
        """
        The VAEGAN model with Negative Binomial distribution as Latent Variable
        """
        super(VAEGAN_NEG_BI, self).__init__()
        self.encoder = encoder 
        self.decoder = decoder
        self.classifier = classifier
        assert(self.encoder.latent_size == self.decoder.input_size)
        assert(self.encoder.latent_size == self.classifier.input_size)
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = (mu + eps*std)
        return z

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decoder(z)
        print(x_hat)
        print(x_hat.shape)
        Negative_binomial = NegativeBinomial(total_count=F.sigmoid(x_hat[:,0]), logits=F.softmax(x_hat[:,1]))
        x_hat = Negative_binomial.sample()
        y_hat = self.classifier(z)
        return x_hat, y_hat, mu, logvar


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()